In [7]:
import design_tools_hermes as dt
import numpy as np
import aux_funcs as aux

#Utilizando para extrair o CD0
# gravity = 9.81
# aircraft = dt.hermes_aircraft()
# new_dimensions = dt.geometry(aircraft)
# aircraft['dimensions'].update(new_dimensions)
# aero, CLmax = dt.aerodynamics(aircraft, Mach = 0.78, altitude = 11000, n_engines_failed = 0, flap_def = 0, 
#                  slat_def = 0, lg_down = 0, h_ground = 0, W0_guess = 97.62e3*gravity)

In [8]:
aircraft_2 = dt.hermes_aircraft()
new_dimensions = dt.geometry(aircraft_2)
aircraft_2['dimensions'].update(new_dimensions)

gravity = 9.81
W0_guess = 957373.425418667
T0_guess = 297517.2708975127

Mach_cruise = 0.78
altitude_cruise = 11000
range_cruise = 6500000

Mach_altcruise = 0.4
range_altcruise = 200*1852
altitude_altcruise = 4572

loiter_time = 2700

altitude_takeoff = 0
distance_takeoff = 2400
TO_flap_def = 0.34906585039887
TO_slat_def = 10*np.pi/180

altitude_landing = 0
distance_landing = 1600
LD_flap_def = 0.69813170079773
LD_slat_def = 20*np.pi/180
MLW_frac = 0.85

W0, Wf, T0, deltaS_wlan, SM_fwd, SM_aft, b_tank_b_w, frac_nlg_fwd, frac_nlg_aft, alpha_tipback, alpha_tailstrike, phi_overturn = dt.analyze(aircraft_2, W0_guess, T0_guess, Mach_cruise, altitude_cruise, range_cruise, Mach_altcruise, range_altcruise, altitude_altcruise, loiter_time, altitude_takeoff, distance_takeoff, TO_flap_def, TO_slat_def, altitude_landing, distance_landing, LD_flap_def, LD_slat_def, MLW_frac)

In [9]:
def create_avl_file(aircraft, x_cg = 'fwd'):
    """
    Create the avl filename
    :params aircraft: dicionario com os parametros do aviao
    :params x_cg: determina se o cg vai ser posterior ou anterior (aft ou fwd)
    """

    #file parameters
    filename=f"{x_cg}.avl"

    #aerofoil parameters
    cl_alpha_perfil = 0.1090*180/2/np.pi**2

    #wing parameters
    wing = aircraft['geo_param']['wing']
    wing_dimensions = aircraft['dimensions']['wing']
    half_wingspan = wing_dimensions['b']/2
    tg_angulo_bordo_ataque = (wing_dimensions['xt'] - wing['xr'])/half_wingspan

    #EH parameters
    eh = aircraft['geo_param']['EH']
    eh_dimensions = aircraft['dimensions']['EH']

    #EV parameters
    ev = aircraft['geo_param']['EV']
    ev_dimensions = aircraft['dimensions']['EV']
    
    with open(filename, "w") as f:
        #Parametros iniciais
        f.write("Hermes\n\n") # Nome da aeronave
        
        f.write("#Mach\n")
        f.write("0.78\n\n")  # Mach de cruzeiro

        f.write("#IYsym   IZsym   Zsym\n")
        f.write("0       0       0.0\n\n")  # Eixos de simetria, definidos como 0

        
        f.write("#Sref    Cref    Bref\n")
        f.write(f"{wing['S']}     {wing_dimensions['cm']}     {wing_dimensions['b']}\n\n")  # Medidas de referêncai

        f.write("#Xref    Yref    Zref\n")
        f.write(f"{aircraft_2['weights'][f'xcg_{x_cg}']}    0.0     0.0\n\n")  # centro de gravidade, pode ser xcg_fwd ou xcg_aft

        f.write("#CDp\n")
        f.write(f"0.01722371386579513\n\n")  #CD0 (arrasto parasita)

        f.write("#====================================================================\n\n")

        #Wing Surface
        f.write("SURFACE\nWing\n\n")
        f.write("#Nchordwise  Cspace   Nspanwise   Sspace\n")
        f.write(f"8.0            1.0       12         1\n\n")  #Parametros de discretizaçao

        f.write("YDUPLICATE\n0.0\n\n")
        # f.write("ANGLE\n0.0\n\n")  #ângulo de inclinação da asa
        f.write("COMPONENT\n1\n\n") 

        # Wing Sections 
        #Cada superfícies de controle deve aparecer em duas seções, onde ela começa e onde ela termina
        #Ainc: é a inclinação da seção a partir da indecêndia da asa
        #Nspanwise e Sspace deixar 0 para discretizar igual a asa
        #Cgain no controle: multiplicador do angulo de deflexão, flap: 1, slat e aileron: -1
        #SgnDup: indica se as superfícies defletem pro mesmo lado nas duas semi asa, aileron: -1, flap e salt: 1
        #Seção 1
        f.write("#-------------------------------------------------------------\n")
        f.write("SECTION\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{wing['xr'] + (half_wingspan*0*tg_angulo_bordo_ataque)}     {half_wingspan*0}     {-wing['zr']}    {aux.chord_surface(aircraft, 0, 1)}    0   0          0\n")
        f.write("AFILE\nairfoil.dat\n") #perfil da seção
        f.write(f"CLAF\n{cl_alpha_perfil}\n") #CL_alpha do perfil/2*pi
        f.write('DESIGN\n')
        f.write('iw 1.0\n\n')

        #Seção 2
        f.write("#-------------------------------------------------------------\n")
        f.write("SECTION\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{wing['xr'] + (half_wingspan*.03*tg_angulo_bordo_ataque)}     {half_wingspan*.03}     {-wing['zr']}    {aux.chord_surface(aircraft, .03, 1)}    0   0          0\n")
        f.write("AFILE\nairfoil.dat\n") #perfil da seção
        f.write(f"CLAF\n{cl_alpha_perfil}\n\n") #CL_alpha do perfil/2*pi
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("flap     1.0   0.7    0.0 0.0 0.0   1.0\n\n")
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("slat  -1.0   -0.1    0.0 0.0 0.0  1.0\n")
        f.write('DESIGN\n')
        f.write('iw 1.0\n\n')


        #Seção 3
        f.write("#-------------------------------------------------------------\n")
        f.write("SECTION\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{wing['xr'] + (half_wingspan*.63*tg_angulo_bordo_ataque)}     {half_wingspan*.63}     {-wing['zr']}    {aux.chord_surface(aircraft, .63, 1)}    0   0          0\n")
        f.write("AFILE\nairfoil.dat\n") #perfil da seção
        f.write(f"CLAF\n{cl_alpha_perfil}\n\n") #CL_alpha do perfil/2*pi
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("flap     1.0   0.7    0.0 0.0 0.0   1.0\n\n")
        f.write('DESIGN\n')
        f.write('iw 1.0\n\n')

        #Seção 4
        f.write("#-------------------------------------------------------------\n")
        f.write("SECTION\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{wing['xr'] + (half_wingspan*.78*tg_angulo_bordo_ataque)}     {half_wingspan*.78}     {-wing['zr']}    {aux.chord_surface(aircraft, .78, 1)}    0   0          0\n")
        f.write("AFILE\nairfoil.dat\n") #perfil da seção
        f.write(f"CLAF\n{cl_alpha_perfil}\n\n") #CL_alpha do perfil/2*pi
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("aileron  -1.0   0.7    0.0 0.0 0.0  -1.0\n")
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("slat  -1.0   -0.1    0.0 0.0 0.0  1.0\n\n")
        f.write('DESIGN\n')
        f.write('iw 1.0\n\n')

        #Seção 5
        f.write("#-------------------------------------------------------------\n")
        f.write("SECTION\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{wing['xr'] + (half_wingspan*1*tg_angulo_bordo_ataque)}     {half_wingspan*1}     {-wing['zr']}    {aux.chord_surface(aircraft, 1, 1)}    0   0          0\n")
        f.write("AFILE\nairfoil.dat\n") #perfil da seção
        f.write(f"CLAF\n{cl_alpha_perfil}\n\n") #CL_alpha do perfil/2*pi
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("aileron  -1.0   0.7    0.0 0.0 0.0  -1.0\n")
        f.write('DESIGN\n')
        f.write('iw 1.0\n\n')


        f.write("#====================================================================\n\n")

        # Empenagem horizontal
        f.write("SURFACE\nH-stab\n\n")
        f.write("#Nchordwise  Cspace   Nspanwise   Sspace\n")
        f.write("6           1.0      6           1.0\n")
        f.write("YDUPLICATE\n0.0\n")
        f.write("COMPONENT\n1\n\n")

        #Seção 1
        f.write("#-------------------------------------------------------------\n\n")
        f.write("SECTION\n\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{eh_dimensions['xr']}     0.0    {-eh['zr']}     {eh_dimensions['cr']}     0.    0          0\n\n")
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("elevator 1.0    0.75     0.0 1.0 0.0  1.0\n")
        f.write("DESIGN\n")
        f.write("it 1.0\n\n")


        #Seção 2
        f.write("#-------------------------------------------------------------\n\n")
        f.write("SECTION\n\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{eh_dimensions['xt']}     {eh_dimensions['yt']}    {eh_dimensions['zt']}     {eh_dimensions['ct']}     0.    0          0\n\n")
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("elevator 1.0    0.75     0.0 1.0 0.0  1.0\n")
        f.write("DESIGN\n")
        f.write("it 1.0\n\n")

        f.write("#====================================================================\n\n")


        # Empenagem vertical
        f.write("SURFACE\nV-stab\n\n")
        f.write("#Nchordwise  Cspace   Nspanwise   Sspace\n")
        f.write("6           1.0      5           1.0\n")
        f.write("COMPONENT\n1\n\n")

        #Seção 1
        f.write("#-------------------------------------------------------------\n\n")
        f.write("SECTION\n\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{ev_dimensions['xr']}     0.0    {-ev['zr']}     {ev_dimensions['cr']}     0.    0          0\n\n")
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("rudder 1.0    0.68     0.0 0.0  1.0 1.0\n\n")

        #Seção 2
        f.write("#-------------------------------------------------------------\n\n")
        f.write("SECTION\n\n")
        f.write("#Xle    Yle    Zle     Chord   Ainc  Nspanwise  Sspace\n")
        f.write(f"{ev_dimensions['xt']}     0.0    {ev_dimensions['zt']}     {ev_dimensions['ct']}     0.    0          0\n\n")
        f.write("CONTROL\n")
        f.write("#Cname   Cgain  Xhinge  HingeVec     SgnDup\n")
        f.write("rudder 1.0    0.68     0.0 0.0  1.0 1.0\n\n")

        f.write("#====================================================================\n\n")

        print("AVL file created successfully.")

create_avl_file(aircraft_2,x_cg = 'fwd')
create_avl_file(aircraft_2,x_cg = 'aft')

AVL file created successfully.
AVL file created successfully.
